# Quantitative Systems Simulator
## Portfolio Risk Analysis with Monte Carlo Simulation

This notebook demonstrates the **QSS** - a hybrid Python/C++ system for quantitative portfolio analysis.

### System Architecture
```
User Input (Portfolio / Dataset)
        |
        v
Python Ingestion Layer (pandas, data cleaning)
        |
        v
+---> C++ Simulation Core (Monte Carlo Engine) <---+
|     - Multi-threaded parallel execution          |
|     - Cholesky decomposition                     |
|     - Variance reduction techniques              |
+--------------------------------------------------+
        |
        v
Statistical Analysis (Python + NumPy/SciPy)
        |
        v
Visualization Dashboard (Matplotlib / Plotly)
        |
        v
JSON + Report Outputs
```

---

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '../python')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from qss import Orchestrator, PortfolioAnalytics, Visualizer
from qss.interface import SimulationConfig, AssetData, HAS_CPP_CORE

# Check if C++ core is available
print(f"C++ Core Available: {HAS_CPP_CORE}")
if HAS_CPP_CORE:
    print("Using high-performance C++ Monte Carlo engine")
else:
    print("Using Python fallback (NumPy-based)")

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

---
## 2. Portfolio Construction

We'll create a **Tech Growth Portfolio** with 5 major assets.

In [ ]:
# Define portfolio assets
portfolio_data = {
    'Symbol': ['AAPL', 'MSFT', 'NVDA', 'GOOG', 'AMZN'],
    'Name': ['Apple', 'Microsoft', 'NVIDIA', 'Alphabet', 'Amazon'],
    'Weight': [0.25, 0.25, 0.20, 0.15, 0.15],
    'Expected Return': [0.12, 0.10, 0.18, 0.11, 0.14],
    'Volatility': [0.25, 0.22, 0.40, 0.28, 0.32]
}

portfolio_df = pd.DataFrame(portfolio_data)
portfolio_df['Weight %'] = portfolio_df['Weight'].apply(lambda x: f"{x:.0%}")
portfolio_df['Expected Return %'] = portfolio_df['Expected Return'].apply(lambda x: f"{x:.1%}")
portfolio_df['Volatility %'] = portfolio_df['Volatility'].apply(lambda x: f"{x:.1%}")

display(portfolio_df[['Symbol', 'Name', 'Weight %', 'Expected Return %', 'Volatility %']])

In [ ]:
# Visualize portfolio allocation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for weights
colors = plt.cm.Set3(np.linspace(0, 1, len(portfolio_df)))
axes[0].pie(portfolio_df['Weight'], labels=portfolio_df['Symbol'], autopct='%1.0f%%',
            colors=colors, explode=[0.02]*5)
axes[0].set_title('Portfolio Allocation', fontsize=14, fontweight='bold')

# Bar chart for risk/return
x = np.arange(len(portfolio_df))
width = 0.35
bars1 = axes[1].bar(x - width/2, portfolio_df['Expected Return']*100, width, 
                    label='Expected Return', color='steelblue')
bars2 = axes[1].bar(x + width/2, portfolio_df['Volatility']*100, width,
                    label='Volatility', color='coral')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_xticks(x)
axes[1].set_xticklabels(portfolio_df['Symbol'])
axes[1].legend()
axes[1].set_title('Risk-Return Profile by Asset', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Correlation Matrix

Asset correlations are crucial for portfolio risk. We define a realistic correlation structure:

In [ ]:
# Define correlation matrix
correlation = np.array([
    [1.00, 0.65, 0.55, 0.60, 0.58],  # AAPL
    [0.65, 1.00, 0.52, 0.58, 0.55],  # MSFT
    [0.55, 0.52, 1.00, 0.48, 0.52],  # NVDA
    [0.60, 0.58, 0.48, 1.00, 0.62],  # GOOG
    [0.58, 0.55, 0.52, 0.62, 1.00]   # AMZN
])

# Compute covariance matrix
vols = portfolio_df['Volatility'].values
cov_matrix = np.outer(vols, vols) * correlation

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            xticklabels=portfolio_df['Symbol'], yticklabels=portfolio_df['Symbol'],
            square=True, linewidths=0.5, ax=ax)
ax.set_title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 3. Monte Carlo Simulation

We'll run **10,000 simulations** over a 1-year horizon (252 trading days).

In [ ]:
# Configuration
config = SimulationConfig(
    n_simulations=10000,
    time_horizon=252,  # Trading days in a year
    risk_free_rate=0.02,  # 2% risk-free rate
    seed=42  # For reproducibility
)

print("Simulation Configuration:")
print(f"  Number of Simulations: {config.n_simulations:,}")
print(f"  Time Horizon: {config.time_horizon} trading days (1 year)")
print(f"  Risk-Free Rate: {config.risk_free_rate:.1%}")

In [ ]:
%%time
# Initialize orchestrator and load portfolio
orch = Orchestrator(config)

# Load portfolio from dictionary
portfolio_dict = {
    "assets": [
        {"symbol": row['Symbol'], "weight": row['Weight'],
         "expected_return": row['Expected Return'], "volatility": row['Volatility']}
        for _, row in portfolio_df.iterrows()
    ]
}
orch.load_portfolio_from_dict(portfolio_dict, cov_matrix)

# Run simulation
print("Running Monte Carlo simulation...")
result = orch.run_simulation()
print(f"Generated {len(result.terminal_values):,} scenarios")

---
## 4. Risk Metrics Analysis

Key metrics for portfolio risk assessment:

In [ ]:
metrics = result.metrics

# Create metrics summary
metrics_data = {
    'Metric': [
        'Expected Annual Return',
        'Volatility (Std Dev)',
        'Sharpe Ratio',
        'Value at Risk (95%)',
        'Value at Risk (99%)',
        'Conditional VaR (95%)',
        'Conditional VaR (99%)',
        'Skewness',
        'Excess Kurtosis',
        'Avg Max Drawdown'
    ],
    'Value': [
        f"{metrics.expected_return:.2%}",
        f"{metrics.volatility:.2%}",
        f"{metrics.sharpe_ratio:.2f}",
        f"{metrics.var_95:.2%}",
        f"{metrics.var_99:.2%}",
        f"{metrics.cvar_95:.2%}",
        f"{metrics.cvar_99:.2%}",
        f"{metrics.skewness:.3f}",
        f"{metrics.kurtosis:.3f}",
        f"{metrics.max_drawdown:.2%}"
    ],
    'Description': [
        'Mean simulated return over 1 year',
        'Standard deviation of returns',
        '(Return - Rf) / Volatility',
        '95% confidence worst-case loss',
        '99% confidence worst-case loss',
        'Expected loss beyond VaR (95%)',
        'Expected loss beyond VaR (99%)',
        'Asymmetry of return distribution',
        'Fat tails (>0 = heavier than normal)',
        'Average peak-to-trough decline'
    ]
}

metrics_df = pd.DataFrame(metrics_data)
display(metrics_df)

---
## 5. Distribution Analysis

In [ ]:
returns = result.terminal_values - 1.0  # Convert to returns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with VaR lines
ax1 = axes[0]
n, bins, patches = ax1.hist(returns, bins=80, density=True, alpha=0.7, 
                             color='steelblue', edgecolor='white')

# Add KDE
from scipy import stats
kde_x = np.linspace(returns.min(), returns.max(), 200)
kde = stats.gaussian_kde(returns)
ax1.plot(kde_x, kde(kde_x), 'darkblue', linewidth=2, label='KDE')

# VaR and CVaR lines
ax1.axvline(metrics.var_95, color='orange', linestyle='--', linewidth=2, 
            label=f'VaR 95%: {metrics.var_95:.1%}')
ax1.axvline(metrics.cvar_95, color='red', linestyle=':', linewidth=2,
            label=f'CVaR 95%: {metrics.cvar_95:.1%}')
ax1.axvline(np.mean(returns), color='green', linestyle='-', linewidth=2,
            label=f'Mean: {np.mean(returns):.1%}')

ax1.set_xlabel('Annual Return', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Portfolio Return Distribution', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')

# Q-Q plot for normality check
ax2 = axes[1]
stats.probplot(returns, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot (Normality Check)', fontsize=14, fontweight='bold')
ax2.get_lines()[0].set_markerfacecolor('steelblue')
ax2.get_lines()[0].set_alpha(0.5)

plt.tight_layout()
plt.show()

---
## 6. Simulation Paths Visualization

In [ ]:
paths = result.simulated_paths
n_steps = paths.shape[1]
time_axis = np.arange(n_steps)

fig, ax = plt.subplots(figsize=(12, 6))

# Plot sample paths
n_sample = 100
sample_idx = np.random.choice(len(paths), n_sample, replace=False)
for idx in sample_idx:
    ax.plot(time_axis, paths[idx], alpha=0.1, color='steelblue', linewidth=0.5)

# Percentile bands
p5 = np.percentile(paths, 5, axis=0)
p25 = np.percentile(paths, 25, axis=0)
p50 = np.percentile(paths, 50, axis=0)
p75 = np.percentile(paths, 75, axis=0)
p95 = np.percentile(paths, 95, axis=0)

ax.fill_between(time_axis, p5, p95, alpha=0.2, color='steelblue', label='5-95%')
ax.fill_between(time_axis, p25, p75, alpha=0.3, color='steelblue', label='25-75%')
ax.plot(time_axis, p50, color='darkblue', linewidth=2, label='Median')

ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Initial Value')

ax.set_xlabel('Trading Days', fontsize=12)
ax.set_ylabel('Portfolio Value', fontsize=12)
ax.set_title(f'Monte Carlo Simulation Paths ({config.n_simulations:,} scenarios)', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.set_xlim(0, n_steps-1)

plt.tight_layout()
plt.show()

---
## 7. Statistical Tests

In [ ]:
# Compute extended analytics
analytics = orch.compute_analytics()

print("=" * 60)
print("STATISTICAL ANALYSIS")
print("=" * 60)

# Distribution analysis
dist = analytics['distribution_analysis']
print("\nDistribution Properties:")
print(f"  Skewness: {dist['moments']['skewness']:.3f}")
print(f"  Excess Kurtosis: {dist['moments']['kurtosis']:.3f}")
print(f"  Is Fat-Tailed: {dist['is_fat_tailed']}")
print(f"  Is Skewed: {dist['is_skewed']}")

# Normality tests
print("\nNormality Tests:")
jb = dist['normality_tests']['jarque_bera']
sw = dist['normality_tests']['shapiro_wilk']
print(f"  Jarque-Bera: statistic={jb['statistic']:.2f}, p-value={jb['p_value']:.4f}")
print(f"  Shapiro-Wilk: statistic={sw['statistic']:.4f}, p-value={sw['p_value']:.4f}")
print(f"  Conclusion: {'Normal' if jb['is_normal'] else 'Non-Normal'} distribution")

# Confidence intervals
print("\n95% Confidence Intervals:")
cis = analytics['confidence_intervals']
for name, ci in cis.items():
    print(f"  {name}: [{ci['lower']:.4f}, {ci['upper']:.4f}]")

---
## 8. VaR Analysis at Multiple Confidence Levels

In [ ]:
# Calculate VaR at multiple levels
confidence_levels = [0.90, 0.95, 0.99, 0.999]
var_values = [np.percentile(returns, (1-cl)*100) for cl in confidence_levels]
cvar_values = [np.mean(returns[returns <= var]) for var in var_values]

var_df = pd.DataFrame({
    'Confidence Level': [f"{cl:.1%}" for cl in confidence_levels],
    'VaR': [f"{v:.2%}" for v in var_values],
    'CVaR (Expected Shortfall)': [f"{v:.2%}" for v in cvar_values]
})

print("Value at Risk Analysis")
print("="*50)
display(var_df)

# Visual
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(returns, bins=80, density=True, alpha=0.7, color='steelblue', edgecolor='white')

colors = ['green', 'orange', 'red', 'darkred']
for cl, var, color in zip(confidence_levels, var_values, colors):
    ax.axvline(var, color=color, linestyle='--', linewidth=2, label=f'VaR {cl:.0%}: {var:.1%}')

ax.set_xlabel('Annual Return', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Value at Risk at Multiple Confidence Levels', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

---
## 9. Convergence Analysis

Monte Carlo estimates converge as the number of simulations increases:

In [ ]:
# Analyze convergence
sample_sizes = np.logspace(2, np.log10(len(returns)), 50).astype(int)

mean_estimates = [np.mean(returns[:n]) for n in sample_sizes]
var_estimates = [np.percentile(returns[:n], 5) for n in sample_sizes]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean convergence
axes[0].plot(sample_sizes, mean_estimates, 'b-', linewidth=2)
axes[0].axhline(np.mean(returns), color='r', linestyle='--', label=f'Final: {np.mean(returns):.4f}')
axes[0].set_xscale('log')
axes[0].set_xlabel('Number of Simulations', fontsize=12)
axes[0].set_ylabel('Mean Return Estimate', fontsize=12)
axes[0].set_title('Mean Estimate Convergence', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# VaR convergence
axes[1].plot(sample_sizes, var_estimates, 'b-', linewidth=2)
axes[1].axhline(metrics.var_95, color='r', linestyle='--', label=f'Final: {metrics.var_95:.4f}')
axes[1].set_xscale('log')
axes[1].set_xlabel('Number of Simulations', fontsize=12)
axes[1].set_ylabel('VaR (95%) Estimate', fontsize=12)
axes[1].set_title('VaR Estimate Convergence', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Convergence Error (SE of mean): {result.convergence_error:.6f}")

---
## 10. Generate Reports

In [ ]:
# Generate JSON report
orch.generate_report('../output/portfolio_analysis.json', format='json')
print("JSON report saved to: ../output/portfolio_analysis.json")

# Generate HTML report
orch.generate_report('../output/portfolio_analysis.html', format='html')
print("HTML report saved to: ../output/portfolio_analysis.html")

---
## Summary

This notebook demonstrated the **Quantitative Systems Simulator**:

### Key Features
1. **Hybrid Python/C++ Architecture** - C++ for performance, Python for ease of use
2. **Monte Carlo Simulation** - Correlated asset returns with Cholesky decomposition
3. **Comprehensive Risk Metrics** - VaR, CVaR, Sharpe ratio, drawdowns
4. **Statistical Analysis** - Normality tests, confidence intervals
5. **Professional Visualization** - Distribution plots, path simulations, heatmaps

### Portfolio Results
| Metric | Value |
|--------|-------|
| Expected Return | {:.1%} |
| Volatility | {:.1%} |
| Sharpe Ratio | {:.2f} |
| VaR (95%) | {:.1%} |
| CVaR (95%) | {:.1%} |

In [ ]:
# Final summary
print("=" * 60)
print("PORTFOLIO SUMMARY")
print("=" * 60)
print(f"Portfolio: Tech Growth ({', '.join(portfolio_df['Symbol'].tolist())})")
print(f"Simulations: {config.n_simulations:,}")
print(f"Time Horizon: 1 Year ({config.time_horizon} trading days)")
print()
print(f"Expected Annual Return: {metrics.expected_return:.1%}")
print(f"Volatility: {metrics.volatility:.1%}")
print(f"Sharpe Ratio: {metrics.sharpe_ratio:.2f}")
print(f"VaR (95%): {metrics.var_95:.1%}")
print(f"CVaR (95%): {metrics.cvar_95:.1%}")
print("=" * 60)